In [1]:
%load_ext autoreload
%autoreload 2

In [2]:
%matplotlib inline 

In [3]:
import pandas as pd
import numpy as np

In [4]:
df_geom = pd.read_csv('../data/geometry_bestguess.hgcal.txt', sep=' ')
df_constr = pd.read_csv('../data/220712_strobbe_elinks-mapping.csv')
showermax_layers = [7,9,11,13]
hcal_start = 27
elinks_showermax = 4
elinks_regular = 2
# no asumption made on the type of STC, assuming STC4 5E+3M everywhere
elinks_stc = 5

In [5]:
print(df_geom.head())
print(df_constr.head())

   plane  u  v itype      x0      y0  irot  nvertices   vx_0   vy_0  ...  \
0      1  2  0   gIe  334.88    0.00     3          6  334.9   96.7  ...   
1      1  2  1   dIe  251.16  145.01     3          5  251.2  241.7  ...   
2      1  3  0    FI  502.32    0.00     0          6  502.3  -96.7  ...   
3      1  3  1    FI  418.60  145.01     1          6  418.6   48.3  ...   
4      1  3  2    FI  334.88  290.02     1          6  334.9  193.3  ...   

   trigPp0_type           dataPp1           trigPp1  dataPp1_type  \
0             A  PP1(CE-)5_FO_1_2  PP1(CE-)3_FO_2_2            Ap   
1             B  PP1(CE-)5_FO_1_2  PP1(CE-)3_FO_2_3            Ap   
2             A  PP1(CE-)5_FO_1_2  PP1(CE-)3_FO_2_2            Ap   
3             A  PP1(CE-)5_FO_1_3  PP1(CE-)3_FO_2_2            Ap   
4             B  PP1(CE-)5_FO_1_2  PP1(CE-)3_FO_2_3            Ap   

   trigPp1_type         dataPp2      DAQ  engine_trig_fibres  \
0            Ap  PPFO(CE-)X5_14  DAQ_2_2                   3   


In [6]:
df_geom = df_geom[['plane','u','v','trigLinks']]
df_constr = df_constr.set_index(['layer', 'u', 'v'])

In [7]:
def get_constr_bcstc(row):
    layer = int(row.plane)
    u = int(row.u)
    v = int(row.v)
    bcstc_elinks = 0
    constr_elinks = 999
    bcstc_elinks = elinks_showermax if layer in showermax_layers else elinks_regular
    # no trigger links in even layers
    if layer%2==0 and layer<hcal_start: bcstc_elinks = 0 
    if layer>=hcal_start: bcstc_elinks = elinks_stc
    layer_tmp = min(layer, hcal_start) # all HCAL layers have the same constraints
    if df_constr.index.isin([(layer_tmp, u, v)]).any():
        constr_elinks = int(df_constr.loc[(layer_tmp,u,v)])
    return min(bcstc_elinks, constr_elinks)

In [8]:
df_geom['trigLinksBCSTC'] = df_geom.apply(get_constr_bcstc, axis=1)

In [9]:
df_geom['ok'] = df_geom['trigLinks']>=df_geom['trigLinksBCSTC']

In [10]:
df_geom.query('ok==False and plane<6')

,plane,u,v,trigLinks,trigLinksBCSTC,ok
35,1,9,0,0.0,2,False
44,1,10,2,0.0,2,False
50,1,10,8,0.0,2,False
132,3,9,0,0.0,2,False
141,3,10,2,0.0,2,False
147,3,10,8,0.0,2,False
229,5,9,0,0.0,2,False
238,5,10,2,0.0,2,False
244,5,10,8,0.0,2,False
3206,1,9,9,0.0,2,False


In [11]:
df_geom.query('ok==False')

,plane,u,v,trigLinks,trigLinksBCSTC,ok
35,1,9,0,0.0,2,False
44,1,10,2,0.0,2,False
50,1,10,8,0.0,2,False
132,3,9,0,0.0,2,False
141,3,10,2,0.0,2,False
...,...,...,...,...,...,...
4450,25,1,10,0.0,2,False
4451,25,8,11,0.0,2,False
4453,25,6,11,0.0,2,False
4454,25,5,11,0.0,2,False
